In [76]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [77]:
source_df = pd.read_excel(r'C:\Users\user\Desktop\DGBF WS\final data.xlsx', index_col=0)

In [78]:
source_df

,"Crude oil, Dubai","Coal, Australian","Natural gas, US","Natural gas, Europe","Liquefied natural gas, Japan",U$/MT,ChatGPT_Score,BERT_Score
Month,,,,,,,,
2013-10-01,106.302273,79.4125,3.6739,11.370000,15.300000,752.368022,-2,0
2013-11-01,105.853810,82.2500,3.6174,11.420000,15.400000,757.827849,-2,1
2013-12-01,107.919524,84.3375,4.2366,11.550000,16.380000,761.290646,-1,0
2014-01-01,104.010000,81.6100,4.7008,11.590000,16.670000,810.300120,-4,0
2014-02-01,104.940000,76.2900,5.9733,11.300000,16.760000,861.274592,-5,0
...,...,...,...,...,...,...,...,...
2022-04-01,102.680000,310.8500,6.5306,32.204187,16.287524,695.031958,-14,-1
2022-05-01,108.320000,371.8600,8.1373,29.170723,16.679091,723.310581,-8,-1
2022-06-01,115.730000,374.0800,7.6740,33.557731,15.533921,762.065675,-9,-1


In [79]:
# 종속변수와 독립변수 나누기
independent_list = ['Crude oil, Dubai', 'Coal, Australian','Natural gas, US','Natural gas, Europe','Liquefied natural gas, Japan', 'ChatGPT_Score', 'BERT_Score']
dependent_list = ['U$/MT']

# 데이터 전처리
X = source_df[independent_list]
y = source_df[dependent_list]
def create_dataset(data, X, y, source_len, time_steps):
    Xs, ys = [], []
    for i in range(len(data) - source_len - time_steps + 1):
        Xs.append(X.iloc[i:i+source_len, :-1])
        ys.append(y.iloc[i + source_len + time_steps - 1, -1])
    return np.array(Xs), np.array(ys)

# 기존 논문에서는 source_len == 100
source_len = 70
time_steps = 3

Xs, ys = create_dataset(source_df, X, y, source_len, time_steps)
type(Xs), type(ys)

(numpy.ndarray, numpy.ndarray)

In [80]:
# Train-test 분리
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.2, random_state=42)

#X_train = X_train.astype(float)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

# 스케일링
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

num_samples, time_steps, num_features = X_train.shape
X_train_reshaped = X_train.reshape(num_samples, time_steps * num_features)
num_samples, time_steps, num_features = X_test.shape
X_test_reshaped = X_test.reshape(num_samples, time_steps * num_features)

X_train_scaled = scaler_X.fit_transform(X_train_reshaped)
X_test_scaled = scaler_X.transform(X_test_reshaped)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

In [81]:
# X_train_scaled의 형태를 확인
print("X_train_scaled shape:", X_test_scaled.shape)

# X_train_scaled의 형태 수정
num_samples, num_time_steps = X_train_scaled.shape
num_time_steps = source_len
num_features = len(independent_list)-1

X_train_scaled = X_train_scaled.reshape(num_samples, num_time_steps, num_features)

# X_test_scaled의 형태 수정
num_samples, num_time_steps = X_test_scaled.shape
num_time_steps = source_len
num_features = len(independent_list)-1
X_test_scaled = X_test_scaled.reshape(num_samples, num_time_steps, num_features)
X_train_scaled, X_test_scaled 

X_train_scaled shape: (7, 420)


(array([[[0.73534132, 0.41594111, 0.45892936, 0.76120567, 0.91445767,
          0.57894737],
         [0.60157942, 0.36046085, 0.56152261, 0.64973136, 0.88594357,
          0.57894737],
         [0.41377608, 0.29703409, 0.40525798, 0.77082795, 0.8873722 ,
          0.52941176],
         ...,
         [0.09227516, 0.00354313, 0.00512456, 0.        , 0.37724167,
          0.28358209],
         [0.18777827, 0.00784388, 0.        , 0.00433649, 0.25809961,
          0.80327869],
         [0.21560552, 0.        , 0.01876006, 0.00553363, 0.17122951,
          0.50746269]],
 
        [[1.        , 0.63594535, 0.65489358, 0.76120567, 0.93726896,
          0.63157895],
         [0.97134123, 0.54245248, 0.54035958, 0.69790958, 0.84971417,
          0.47368421],
         [0.92396   , 0.44090305, 0.51088585, 0.6809821 , 0.89892377,
          0.76470588],
         ...,
         [0.45069012, 0.06895763, 0.08747297, 0.05042455, 0.36027017,
          0.        ],
         [0.3477293 , 0.066313  , 0.061

In [82]:
model = Sequential()
model.add(LSTM(10, activation='relu', input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.fit(X_train_scaled, y_train_scaled, epochs=1000, batch_size=140, verbose=1)

Epoch 1/1000
1/1 [==============================] - 1s 796ms/step - loss: 0.2668
Epoch 2/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.2580
Epoch 3/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.2496
Epoch 4/1000
1/1 [==============================] - 0s 10ms/step - loss: 0.2416
Epoch 5/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.2340
Epoch 6/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.2266
Epoch 7/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.2195
Epoch 8/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.2128
Epoch 9/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.2064
Epoch 10/1000
1/1 [==============================] - 0s 13ms/step - loss: 0.2002
Epoch 11/1000
1/1 [==============================] - 0s 11ms/step - loss: 0.1944
Epoch 12/1000
1/1 [==============================] - 0s 12ms/step - loss: 0.1888
Epoch 13/1000
1/1 [=================

In [83]:
def calculate_r2_adj(y_true, y_pred, n_features):
    """
    Calculate adjusted R-squared (R2(adj)).

    Parameters:
        y_true (array-like): 실제값들을 담은 배열 또는 리스트
        y_pred (array-like): 예측값들을 담은 배열 또는 리스트
        n_features (int): 독립 변수의 수

    Returns:
        float: R2(adj) 값
    """
    assert len(y_true) == len(y_pred), "실제값과 예측값의 길이가 동일해야 합니다."

    # R-squared 계산
    total_sum_of_squares = sum((true_val - np.mean(y_true))**2 for true_val in y_true)
    residual_sum_of_squares = sum((true_val - pred_val)**2 for true_val, pred_val in zip(y_true, y_pred))
    r_squared = 1 - (residual_sum_of_squares / total_sum_of_squares)

    # R2(adj) 계산
    n = len(y_true)
    r2_adj = 1 - (1 - r_squared) * (n - 1) / (n - n_features - 1)

    return r2_adj

In [84]:
# 테스트 데이터로 예측 수행
y_pred = model.predict(X_test_scaled)
y_pred

1/1 [==============================] - 0s 116ms/step


array([[0.7522726 ],
       [0.02326041],
       [0.35833886],
       [0.20168272],
       [0.19906521],
       [0.53853095],
       [0.34882256]], dtype=float32)

In [85]:
# 스케일링 복원
y_test_predicted = scaler_y.inverse_transform(y_pred)
y_test_predicted

array([[967.06854],
       [285.10892],
       [598.5604 ],
       [452.01532],
       [449.56674],
       [767.1223 ],
       [589.6583 ]], dtype=float32)

In [86]:
# MAPE 계산
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

test_mape = mape(y_test, y_test_predicted)
print('Test MAPE:', test_mape)

test_r2 = calculate_r2_adj(y_test, y_test_predicted, len(independent_list))
print('R2(adj): ', test_r2)

Test MAPE: 16.3484558031808
R2(adj):  [2.53716583]


In [87]:
import matplotlib.pyplot as plt
#test_mape = mape(y_test_actual, y_test_predicted)
plt.plot(y_test, label='actual')
plt.plot(y_test_predicted, label='prediction')
plt.legend()
plt.show()
     

In [88]:
# 전체로 예측 수행
## reshape로 자료형 편집
num_samples, time_steps, num_features = Xs.shape
Xs_reshaped = Xs.reshape(num_samples, time_steps * num_features)
Xs_scaled = scaler_X.fit_transform(Xs_reshaped)

num_samples, num_time_steps = Xs_scaled.shape
num_time_steps = source_len
num_features = len(independent_list)-1
Xs_scaled = Xs_scaled.reshape(num_samples, num_time_steps, num_features)
ys = ys.reshape(-1, 1)

ys_pred = model.predict(Xs_scaled)
ys_predicted = scaler_y.inverse_transform(ys_pred)

2/2 [==============================] - 0s 5ms/step


In [89]:
# MAPE 계산
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

test_mape = mape(ys, ys_predicted)
print('Test MAPE:', test_mape)

test_r2 = calculate_r2_adj(ys, ys_predicted, len(independent_list))
print('R2(adj): ', test_r2)

Test MAPE: 6.844934430254504
R2(adj):  [0.92272168]


In [90]:
plt.plot(ys, label='actual')
plt.plot(ys_predicted, label='prediction')
plt.legend()
plt.show()